# Importando as bibliotecas e logando no drive:

In [1]:
# Importando as bibliotecas necessárias:
import numpy as np
import tensorflow as tf
import cv2
import os
import time
from google.colab import drive

# Conectando ao Google Drive:
drive.mount('/content/drive')

Mounted at /content/drive


# Funções utilizadas:

In [2]:
# Carrega o modelo e suas informações da memória:
def preparar_modelo(local):
  # Carregar o modelo tflite escolhido da memória:
  with open(local, 'rb') as f:
      tflite_model = f.read()

  # Instanciar um Interpretador
  try:
      interpreter = tf.lite.Interpreter(model_content=tflite_model)
      interpreter.allocate_tensors()
  except Exception as e:
      print("Erro ao carregar o modelo:", e)
  else:
      print("Modelo carregado com sucesso.\n")

  # Preparar as Entradas:
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  input_shape = input_details[0]['shape']

  return interpreter, input_details, output_details, input_shape

In [3]:
# Função para aplicar resize e normalizar a imagem para a entrada:
def processamento_da_imagem(imagem, escala_x, escala_y):
  # Redimensionar a imagem para as dimensões esperadas pelo modelo
  resized_image = cv2.resize(imagem, (escala_x, escala_y))

  # Converter de BGR para RGB (OpenCV carrega imagens como BGR por padrão)
  new_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

  # Converter para ponto flutuante e normalizar
  image_norm = new_image.astype(np.float32) / 255.0

  # Transpor as dimensões para corresponder à forma esperada pelo modelo
  image_norm = image_norm.transpose(2, 0, 1)

  # Adicionar uma dimensão extra para corresponder à forma esperada pelo modelo
  image_norm = np.expand_dims(image_norm, axis=0)

  return image_norm, resized_image

In [4]:
# Função para realizar inferência com o modelo tflite:
def inferencia(image_norm, input_details, output_details, interpreter):
  # Definir os dados de entrada do modelo
  interpreter.set_tensor(input_details[0]['index'], image_norm)

  # Realizando a inferência e computando o tempo decorrido
  start = time.time()
  interpreter.invoke()
  end = time.time()
  inference_time = end-start

  # Obtendo os resultados da inferência
  output_data = interpreter.get_tensor(output_details[0]['index'])

  # Filtrando as linhas onde a probabilidade é significativa:
  filtered_output_data = [detection for detection in output_data if detection[6] > 0.01]

  return filtered_output_data, inference_time

In [5]:
def calcular_iou(caixa1, caixa2):
    # Extrair coordenadas das caixas delimitadoras
    x1_c1, y1_c1, largura_c1, altura_c1, _ = caixa1
    x1_c2, y1_c2, largura_c2, altura_c2, _ = caixa2

    # Calcular coordenadas dos vértices opostos
    x2_c1 = x1_c1 + largura_c1
    y2_c1 = y1_c1 + altura_c1
    x2_c2 = x1_c2 + largura_c2
    y2_c2 = y1_c2 + altura_c2

    # Calcular coordenadas da interseção
    x1_intersecao = max(x1_c1, x1_c2)
    y1_intersecao = max(y1_c1, y1_c2)
    x2_intersecao = min(x2_c1, x2_c2)
    y2_intersecao = min(y2_c1, y2_c2)

    # Calcular área de interseção
    area_intersecao = max(0, x2_intersecao - x1_intersecao + 1) * max(0, y2_intersecao - y1_intersecao + 1)

    # Calcular áreas das caixas delimitadoras
    area_caixa1 = (x2_c1 - x1_c1 + 1) * (y2_c1 - y1_c1 + 1)
    area_caixa2 = (x2_c2 - x1_c2 + 1) * (y2_c2 - y1_c2 + 1)

    # Calcular área da união
    area_uniao = area_caixa1 + area_caixa2 - area_intersecao

    # Calcular IoU
    iou = area_intersecao / area_uniao

    return iou

In [6]:
# Escreve as boxes na imagem e salva num caminho fixo
def escreve_imagem_erro(img, lista_deteccoes, lista_rotulos, idx, tipo):
  # Instânciando a variável fora do if
  caminho_de_salvamento = 'none'

  # Definindo o caminho de salvar o dado atual:
  if tipo == 'erro':
    caminho_de_salvamento = '/content/drive/MyDrive/yolo_pesos/falhas'
  elif tipo == 'acerto':
    caminho_de_salvamento = '/content/drive/MyDrive/yolo_pesos/acertos'

  # Verifica se o diretório de salvamento existe, caso contrário, cria
  if not os.path.exists(caminho_de_salvamento):
      os.makedirs(caminho_de_salvamento)

  # Itera sobre as lista de boxes e desenha cada uma na imagem
  if len(lista_deteccoes) > 0:
    for box in lista_deteccoes:
      # Supondo que cada box seja uma tupla (x, y, largura, altura)
      x, y, w, h, classe = box
      # Desenha o retângulo na imagem
      img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      img = cv2.putText(img, f'detectado {classe}', (x, y + h + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,  (255, 0, 0), 1)

  if len(lista_rotulos) > 0:
    for box in lista_rotulos:
      # Supondo que cada box seja uma tupla (x, y, largura, altura)
      x, y, w, h, classe = box
      # Desenha o retângulo na imagem
      img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
      img = cv2.putText(img, f'rotulo {classe}', (x, y + h + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,  (0, 255, 0), 1)

  # Define o nome do arquivo de saída
  caminho_completo = os.path.join(caminho_de_salvamento,  'exemplo_' + str(idx) + '.jpg')

  # Salva a imagem com as boxes desenhadas
  cv2.imwrite(caminho_completo, img)

In [7]:
def teste_modelo(diretorio_images, diretorio_labels, interpreter, input_details, output_details, input_shape, classes):
  # Listar todos os arquivos no diretório
  images = os.listdir(diretorio_images)
  labels = os.listdir(diretorio_labels)

  # Definindo as escalas das imagens de entrada
  escala_x = int(input_shape[2])
  escala_y = int(input_shape[3])

  # Dicionário que guarda as métricas
  metricas_classes = {}

  # Inicializando as métricas das classes
  for i in range(len(classes)):
    metricas_classes[classes[i]] = {'TP': 0, 'FP': 0, 'FN': 0}

  # Contagem de imagens exemplo:
  imagens_erro_exemplo = 0
  imagens_acerto_exemplo = 0

  # Lista com todos os tempos de inferência:
  tempos_de_inferencia = []

  # Verificar se o número de imagens e rótulos é o mesmo
  if len(images) != len(labels):
      print("O número de imagens e rótulos não corresponde.")
  else:
      # Loop sobre as imagens e rótulos ao mesmo tempo
      for imagem_nome, label_nome in zip(images, labels):
          # Caminhos completos para a imagem e o rótulo
          caminho_imagem = os.path.join(diretorio_images, imagem_nome)
          caminho_label = os.path.join(diretorio_labels, label_nome)

          # Verificar se ambos são arquivos
          if os.path.isfile(caminho_imagem) and os.path.isfile(caminho_label):
              # Carregar a imagem usando OpenCV
              imagem = cv2.imread(caminho_imagem)

              # Verificar se a imagem foi carregada corretamente
              if imagem is not None:
                  # Processando a imagem:
                  img_norm, img_resized = processamento_da_imagem(imagem, escala_x, escala_y)

                  # Realizando a inferência:
                  output, tempo_decorrido = inferencia(img_norm, input_details, output_details, interpreter)
                  tempos_de_inferencia.append(tempo_decorrido)

                  # Listas de detecções e rotulos da imagem atual:
                  deteccoes = []
                  rotulos = []

                  for detection in output:
                      # Obter os valores das coordenadas da bounding box:
                      _, x1, y1, x2, y2, classe_pred, probabilidade_pred = detection

                      # Corrigindo problemas de coordenadas fora da imagem:
                      x1 = max(int(x1), 0)
                      y1 = max(int(y1),0)
                      x2 = min(int(x2), escala_x)
                      y2 = min(int(y2), escala_y)

                      # Calcular as dimensões da bounding box:
                      bbox_largura = x2 - x1
                      bbox_altura = y2 - y1

                      # Salva a detecção atual na lista de detecções:
                      deteccoes.append([x1, y1, int(bbox_largura), int(bbox_altura), int(classe_pred)])

                  # Ler os rotulos do arquivo .txt:
                  with open(caminho_label, 'r') as file:
                    for linha in file:
                        label_content = linha.strip()  # Remover espaços em branco extras
                        valores = label_content.split() # Dividir o conteúdo em uma lista de valores

                        # Converter os valores para os tipos apropriados
                        classe = int(valores[0])
                        centro_x = int(escala_x * float(valores[1]))
                        centro_y = int(escala_y * float(valores[2]))
                        largura = int(escala_x * float(valores[3]))
                        altura = int(escala_y * float(valores[4]))
                        x = centro_x - int(largura/2)
                        y = centro_y - int(altura/2)

                        # Salva a label atual na lista de rotulos:
                        rotulos.append([int(x), int(y), int(largura), int(altura), int(classe)])

                  # Cria cópias das listas:
                  temp_detections = deteccoes.copy()
                  temp_labels = rotulos.copy()

                  # Verficação se ouver algum match
                  img_match = 0

                  # Realizando o matching das bbox e suas classificações:
                  for detection in deteccoes:
                    for label in rotulos:
                      iou = calcular_iou(detection, label)
                      if (iou > 0.5):
                        # Verifica se a classe é igual:
                        if (detection[4] == label[4]):
                          img_match = 1
                          metricas_classes[classes[detection[4]]]['TP'] += 1

                          # Remove essas da lista pq são do mesmo objeto
                          if detection in temp_detections:
                            temp_detections.remove(detection)
                          if label in temp_labels:
                            temp_labels.remove(label)


                  # Selecionando images que contem apenas erros:
                  if img_match == 0 and len(temp_detections) > 0 or len(temp_labels) > 0:
                    if imagens_erro_exemplo % 15 == 0:
                      escreve_imagem_erro(img_resized, temp_detections, temp_labels, imagens_erro_exemplo, 'erro')
                    imagens_erro_exemplo += 1

                  # Selecionando images que contem apenas acertos:
                  if img_match == 1 and len(temp_detections) == 0 and len(temp_labels) == 0:
                    if imagens_acerto_exemplo % 35 == 0:
                      escreve_imagem_erro(img_resized, deteccoes, rotulos, imagens_acerto_exemplo, 'acerto')
                    imagens_acerto_exemplo += 1

                  # Acessando cada detecção errada e associando a sua devida classe:
                  for detection in temp_detections:
                    metricas_classes[classes[detection[4]]]['FP'] += 1
                  for label in temp_labels:
                    metricas_classes[classes[label[4]]]['FN'] += 1

              else:
                print(f"Não foi possível carregar a imagem: {caminho_imagem}")
          else:
            print(f"Arquivos ausentes para a imagem {imagem_nome} ou rótulo {label_nome}.")

  # Calculando as métricas do nosso teste:
  num_imgs = len(images)
  tempos_de_inferencia_medio = np.mean(tempos_de_inferencia)
  tempo_inferencia_min = min(tempos_de_inferencia)
  tempo_inferencia_max = max(tempos_de_inferencia)

  # Exibindo as métricas e detalhes do teste:
  print("Informações gerais do teste:")
  print(f"Imagens inferidas: {num_imgs}")
  print(f"Tempo de inferência Minimo: {tempo_inferencia_min:.2f}, Médio: {tempos_de_inferencia_medio:.2f}, Máximo: {tempo_inferencia_max:.2f}")
  print()
  for i in range(len(classes)):
      TP = metricas_classes[classes[i]]['TP']
      FP = metricas_classes[classes[i]]['FP']
      FN = metricas_classes[classes[i]]['FN']
      precisao = TP / (TP + FP) if (TP + FP) > 0 else 0
      recall = TP / (TP + FN) if (TP + FN) > 0 else 0
      f1_score = 2 * (precisao * recall) / (precisao + recall) if (precisao + recall) > 0 else 0
      print(f"Métricas para a classe {classes[i]}:")
      print(f"True positives: {TP}, False positives: {FP}, False negatives: {FN}")
      print(f"Precisão: {precisao:.2f}")
      print(f"Recall: {recall:.2f}")
      print(f"Pontuação F1: {f1_score:.2f}")
      print()

# Realizando o teste:

In [8]:
# Diretório contendo as imagens
diretorio_images = '/content/drive/MyDrive/yolo_pesos/testes/testes/test_booleano/images'
diretorio_labels = '/content/drive/MyDrive/yolo_pesos/testes/testes/test_booleano/labels'

# Definindo onde está o nosso modelo:
local_modelo_tflite = '/content/drive/MyDrive/yolo_pesos/booleano.tflite'

# Preparando nosso modelo para o teste:
modelo, input_details, output_details, input_shape = preparar_modelo(local_modelo_tflite)

# Array das classes na ordem do treinamento:
classes = ['benigno','maligno']

# Testando:
teste_modelo(diretorio_images, diretorio_labels, modelo, input_details, output_details, input_shape, classes)

Modelo carregado com sucesso.

Informações gerais do teste:
Imagens inferidas: 205
Tempo de inferência Minimo: 1.01, Médio: 1.11, Máximo: 2.17

Métricas para a classe benigno:
True positives: 83, False positives: 26, False negatives: 27
Precisão: 0.76
Recall: 0.75
Pontuação F1: 0.76

Métricas para a classe maligno:
True positives: 83, False positives: 18, False negatives: 24
Precisão: 0.82
Recall: 0.78
Pontuação F1: 0.80

